In [ ]:
# Final Project

In [ ]:
# foodpanda!!!!!!!

In [107]:
# foodpanda
import requests as rq
from pyquery import PyQuery as pq
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 輸入外送地址
address = input("輸入外送地址(OO市OO區OO路OO號):")

# selenium 開啟瀏覽器
driver = webdriver.Chrome()
driver.get("https://www.foodpanda.com.tw/")
# driver.maximize_window()

#找到輸入框
element = driver.find_element_by_xpath("//*[@id='delivery-information-postal-index']")

#輸入內容
element.send_keys(address)

time.sleep(1)

#提交表單
python_button = driver.find_elements_by_xpath("//*[@id='delivery-information-postal-index-form']/div[2]/button[1]")[0]
python_button.click()

time.sleep(1)

#分頁
# python_button = driver.find_element_by_xpath("//*[@id='filters-react-root']/aside/div[1]")
# python_button.click()

# element = driver.find_element_by_xpath("//*[@id='filters-react-root']/aside/div[1]/div/input")
# element.send_keys(restaurant)
# element.send_keys(Keys.ENTER)

time.sleep(1)

#抓取本頁URL並關閉瀏覽器
html = driver.find_element_by_css_selector("*").get_attribute("outerHTML")
doc = pq(html)

driver.close()

# 地址例外處理
if (doc("section.vendor-list-section.opened > ul > li").text()==""):
    print("----------------------------------------------------------------------------------------------")
    print("Oops! Something Wrong!\nTry to Key in the 'detail'(完整的) address!")

輸入外送地址(OO市OO區OO路OO號):台北市大安區辛亥路三段188號


In [108]:
# 找目前有開放訂購的餐廳
foodList = []
foodCategory = []
foodDict_By_Category = {}
foodDict_By_Stars = {}
foodDict_By_Fee = {}
foodDict_By_Attr = {}


# 處理資料(EX:Name,Rate,Fee...)
for eachStore in doc("section.vendor-list-section.opened > ul > li").items():  
    # {Name} = foodCategory
    foodDict = {}
    foodCategory.append(eachStore("li.vendor-characteristic span").text())
    foodDict[eachStore("span.name").text()] = eachStore("li.vendor-characteristic span").text()
    foodDict_By_Category.update(foodDict)
    # [Name] -> foodList
    foodList.append(eachStore("span.name").text())
    # {Name} = Rate
    foodDict = {}
    foodDict[eachStore("span.name").text()] = eachStore("span.rating strong").text()
    foodDict_By_Stars.update(foodDict)
    # {Name} = Fee
    foodDict = {}
    foodDict[eachStore("span.name").text()] = eachStore("ul.extra-info span").text().split(" ")[0].split("$")[-1]+"$ 運費"
    foodDict_By_Fee.update(foodDict)
    # {Name} = attr
    foodAttr = {}
    foodAttr[eachStore("span.name").text()] = eachStore("a").attr("href")
    foodDict_By_Attr.update(foodAttr)  
    
# 由值找鍵的函式   
def get_key (dict, value):
    return [k for k, v in dict.items() if v == value]

# 沒有星星 -> 0
NoStars = []
NoStars.append(get_key(foodDict_By_Stars,""))
if NoStars[0] != []:
    for i in range(len(NoStars[0])):
        foodDict_By_Stars[NoStars[0][i]] = "0"  

# 星星字串 -> 星星數量(float)
for i in range(len(foodDict_By_Stars)):
    foodDict_By_Stars[foodList[i]] = float(foodDict_By_Stars[foodList[i]])

In [120]:
# 輸入想吃的種類
print("=="*20+" 以下是目前有開放訂購的餐廳 "+"=="*20)
print()

category = input("輸入想吃的食物種類(EX:美式、日式、義式...): ")
print()
print("\033[1;31m =================== 目前開放訂購的餐廳中種類包含「"+category+"」的店家(依星星數量由多至少排序) =====================  \033[0m \n")

webURL = "https://www.foodpanda.com.tw"
results = []
Results_By_Stars ={}
category_name = []
for i in range(len(foodCategory)):
    if category in foodCategory[i]:
        # {Results_Name} = Results_rate
        category_name = get_key(foodDict_By_Category,foodCategory[i])
        if len(category_name)==1:
            Results_By_Stars[str(category_name[0])] = foodDict_By_Stars[category_name[0]]
        else:
            for j in range(len(category_name)):
                Results_By_Stars[category_name[j]] = foodDict_By_Stars[category_name[j]]
    else:
        results.append(-1)
# 例外抓取
if len(results) == len(foodCategory):
    print("可惜! 沒有包含此種類的店家!")        

# 想吃的種類中依照星星數量排序
# [Name,rate]
Results_Sorted_By_Stars = sorted(Results_By_Stars.items(),key = lambda x:x[1],reverse = True)
        
for i in range(len(Results_Sorted_By_Stars)):
    if Results_Sorted_By_Stars[i][1] == 0:
        output = "{}. \033[7;30m {} \033[0m \033[7;32m {} \033[0m  \033[7;33m {}顆星(No Comment) \033[0m ".format(i+1,Results_Sorted_By_Stars[i][0],foodDict_By_Category[Results_Sorted_By_Stars[i][0]],Results_Sorted_By_Stars[i][1])
    else:
        output = "{}. \033[7;30m {} \033[0m \033[7;32m {} \033[0m  \033[7;33m {}顆星 \033[0m ".format(i+1,Results_Sorted_By_Stars[i][0],foodDict_By_Category[Results_Sorted_By_Stars[i][0]],Results_Sorted_By_Stars[i][1])
    print(output+"\n")
    print(webURL+foodDict_By_Attr[Results_Sorted_By_Stars[i][0]])
    print("-"*90)

======================================== 以下是目前有開放訂購的餐廳 ========================================

輸入想吃的食物種類(EX:美式、日式、義式...): 美式

 =================== 目前開放訂購的餐廳中種類包含「美式」的店家(依星星數量由多至少排序) =====================   

1.  STARBUCKS 星巴克 (敦和門市)   店內價 美式料理 飲料    4.8顆星  

https://www.foodpanda.com.tw/restaurant/y1al/starbucks-xing-ba-ke-dun-he-men-shi
------------------------------------------------------------------------------------------
2.  The Shed   店內價 漢堡 美式料理    4.8顆星  

https://www.foodpanda.com.tw/restaurant/w4eh/the-shed
------------------------------------------------------------------------------------------
3.  Pizza Hut必勝客 (和東外送店)   店內價 Pizza 比薩 美式料理    4.7顆星  

https://www.foodpanda.com.tw/restaurant/y0ij/pizza-hutbi-sheng-ke-he-dong-wai-song-dian
------------------------------------------------------------------------------------------
4.  胖老爹美式炸雞 (復興南路店)   炸雞 非店內價 美式料理    4.6顆星  

https://www.foodpanda.com.tw/restaurant/y8qr/pang-lao-die-mei-shi-zha-ji-fu-xing-nan-lu-dian
-------

In [121]:
# 輸入外送餐廳名稱
restaurant = input("輸入想外送的餐廳名稱:")
print()

# 排列最接近的Name
print("\033[1;31m =============== 目前開放訂購的餐廳中最接近「"+restaurant+"」的店家(依星星數量由多至少排序) ===================  \033[0m \n")

foodCloserList = []
for i in range(len(foodList)):
    if restaurant in foodList[i]:
        foodCloserList.append(foodList[i])
        
# 最接近Name的餐廳的星星數量排列
foodCloserDict_By_stars = {}
for i in range(len(foodCloserList)):
    foodCloserDict_By_stars[foodCloserList[i]] = foodDict_By_Stars[foodCloserList[i]]
foodCloserList_By_stars = sorted(foodCloserDict_By_stars.items(),key = lambda x:x[1],reverse = True)

if foodCloserList == []:
    print("可惜! "+restaurant+"目前沒有提供foodpanda外送服務!")
#     print("----------------------------------------------------------------------")
#     # 找出最接近的前n個Name
#     num = input("輸入前N個最接近"+restaurant+"的結果:")
#     print()
#     for i in range(int(num)):
#         foodCloserList.append(foodList[i]) 
#         output = "{:<1d}.\t{:<20s}\t{:>10s}".format(i+1,foodCloserList[i],foodDict_By_Stars[foodList[i]])
#         print(output)

else:
    for i in range(len(foodCloserList)):
        if len(foodCloserList) == 1:
            output = "\033[7;30m {} \033[0m \033[7;33m {}顆星 \033[0m\n\n {} ".format(foodCloserList_By_stars[i][0],foodCloserList_By_stars[i][1],webURL+foodDict_By_Attr[foodCloserList_By_stars[i][0]])
            print(output)
        else:
            output = "{}. \033[7;30m {} \033[0m \033[7;33m {}顆星 \033[0m".format(i+1,foodCloserList_By_stars[i][0],foodCloserList_By_stars[i][1])
            print(output)
            print()
            print(webURL+foodDict_By_Attr[foodCloserList_By_stars[i][0]])
            print("-"*90)

輸入想外送的餐廳名稱:牛排

 =============== 目前開放訂購的餐廳中最接近「牛排」的店家(依星星數量由多至少排序) ===================   

1.  人从众厚切牛排館 (台北安居店)   4.6顆星 

https://www.foodpanda.com.tw/restaurant/w1vm/ren-cong-zhong-hou-qie-niu-pai-guan-tai-bei-da-an-dian-m
------------------------------------------------------------------------------------------
2.  澳客炙燒牛排 (台北和平店)   4.4顆星 

https://www.foodpanda.com.tw/restaurant/w4jr/ao-ke-zhi-shao-niu-pai-tai-bei-he-ping-dian-m
------------------------------------------------------------------------------------------


In [122]:
# 依照星星數量排列(多到少)
print("\033[1;31m ================================= 外送地址附近目前有開放訂購的餐廳(依星星數量由多至少排序) ============================== \033[0m \n")
foodDict_Sorted_By_Stars = sorted(foodDict_By_Stars.items(),key = lambda x:x[1],reverse = True)

# 輸出排序後的店家
for i in range(len(foodDict_Sorted_By_Stars)):
    if foodDict_Sorted_By_Stars[i][1] == 0:
        output = "{:<d}. \033[7;30m {} \033[0m  \033[7;33m {}顆星(No Comment) \033[0m".format(i+1,foodDict_Sorted_By_Stars[i][0],foodDict_Sorted_By_Stars[i][1])
    else:
        output = "{:<d}. \033[7;30m {} \033[0m  \033[7;33m {}顆星 \033[0m".format(i+1,foodDict_Sorted_By_Stars[i][0],foodDict_Sorted_By_Stars[i][1])
    print(output)
    print()
    print(webURL+foodDict_By_Attr[foodDict_Sorted_By_Stars[i][0]])
    print("-"*100)

 ================================= 外送地址附近目前有開放訂購的餐廳(依星星數量由多至少排序) ==============================  

1.  和平海王子海鮮粥    5.0顆星 

https://www.foodpanda.com.tw/restaurant/a6rh/he-ping-hai-wang-zi-hai-xian-zhou
----------------------------------------------------------------------------------------------------
2.  STARBUCKS 星巴克 (敦和門市)    4.8顆星 

https://www.foodpanda.com.tw/restaurant/y1al/starbucks-xing-ba-ke-dun-he-men-shi
----------------------------------------------------------------------------------------------------
3.  The Shed    4.8顆星 

https://www.foodpanda.com.tw/restaurant/w4eh/the-shed
----------------------------------------------------------------------------------------------------
4.  艋舺雞排 (大安通化店)    4.7顆星 

https://www.foodpanda.com.tw/restaurant/y9ek/meng-jia-ji-pai-da-an-tong-hua-dian
----------------------------------------------------------------------------------------------------
5.  Pizza Hut必勝客 (和東外送店)    4.7顆星 

https://www.foodpanda.com.tw/restaurant/y0ij/pizza-hut

In [123]:
# 依照運費排列(少到多)
print("\033[1;31m ================================== 外送地址附近目前有開放訂購的餐廳(依照運費由少至多排序) ================================== \033[0m \n")
foodDict_Sorted_By_Fee = sorted(foodDict_By_Fee.items(),key = lambda x:x[1],reverse = False)

# 輸出排序後的店家
for i in range(len(foodDict_Sorted_By_Stars)):
    output = "{:<d}. \033[7;30m {} \033[0m   \033[7;34m {} \033[0m".format(i+1,foodDict_Sorted_By_Fee[i][0],foodDict_Sorted_By_Fee[i][1])
    print(output)
    print()
    print(webURL+foodDict_By_Attr[foodDict_Sorted_By_Stars[i][0]])
    print("-------------------------------------")

 ================================== 外送地址附近目前有開放訂購的餐廳(依照運費由少至多排序) ==================================  

1.  ZAC ZAG 一楽炸雞     29$ 運費 

https://www.foodpanda.com.tw/restaurant/a6rh/he-ping-hai-wang-zi-hai-xian-zhou
-------------------------------------
2.  STARBUCKS 星巴克 (敦和門市)     29$ 運費 

https://www.foodpanda.com.tw/restaurant/y1al/starbucks-xing-ba-ke-dun-he-men-shi
-------------------------------------
3.  廟口黑客滷味     29$ 運費 

https://www.foodpanda.com.tw/restaurant/w4eh/the-shed
-------------------------------------
4.  通化街米粉湯50年老店(胡記復興旗艦店)     29$ 運費 

https://www.foodpanda.com.tw/restaurant/y9ek/meng-jia-ji-pai-da-an-tong-hua-dian
-------------------------------------
5.  麵工坊 MGF Pasta (台北大安店)     29$ 運費 

https://www.foodpanda.com.tw/restaurant/y0ij/pizza-hutbi-sheng-ke-he-dong-wai-song-dian
-------------------------------------
6.  麵工坊 (公館JR店)     29$ 運費 

https://www.foodpanda.com.tw/restaurant/y3rd/he-ma-ma-shou-zuo-ju-ruo
-------------------------------------
7.  沙巴巴中東美食sababa 

In [124]:
# 選定某家餐廳 
chooseRest = input("輸入確定訂購的餐廳名稱: ")
print()
if chooseRest not in foodList:
    print(" \033[1;31m OOPs! 找不到「"+chooseRest+"」，請輸入餐廳完整名稱! \033[0m")
else:
    print("\033[1;31m ================================== 列出訂購的餐廳資訊 ================================== \033[0m \n")

    # 列出該家資訊(名稱、評價、超連結、運費、菜單、價格)
    StoreName = chooseRest
    StoreRate = foodDict_By_Stars[StoreName]
    StoreFee = foodDict_By_Fee[StoreName]
    StoreAttr = foodDict_By_Attr[StoreName]
    StoreCategory = foodDict_By_Category[StoreName]
    print("餐廳名稱: \033[7;30m {} \033[0m\n".format(StoreName))
    print("類型: \033[7;32m {} \033[0m\n".format(StoreCategory))
    print("評價: \033[7;33m {}顆星 \033[0m\n".format(StoreRate))
    print("運費: \033[7;34m {} \033[0m\n".format(StoreFee))
    print("超連結: {}\n".format(webURL+StoreAttr))

    # 進入該店家的網頁
    RestResponse = rq.get(webURL+StoreAttr)
    RestDoc = pq(RestResponse.text)

    # 爬取菜單，價格
    print("=="*25+" 菜單 "+"=="*25)
    print()

    orderList = []
    nameList = []
    descripList = []
    priceList = []
    orderDict = {}
    
# 處理資料
    for each in RestDoc("div.menu__list > section > div.menu__items > ul > li").items():
        
        orderDict[each(".dish-info > h3 > span").text()] = [each(".dish-info p").text(),each("section > span").text()]
        orderList.append(orderDict)
        
        nameList.append(each(".dish-info > h3 > span").text())
        descripList.append(each(".dish-info p").text())
        priceList.append(each("section > span").text())
        
    if "提供" in nameList[0] :
        del nameList[0]
        del descripList[0]
        del priceList[0]

    # 輸出資料
    for i in range(len(nameList)):
        if descripList[i] == "":
            descripList[i] = "無"
        print("商品名稱: \033[7;30m {} \033[0m\n".format(nameList[i]))
        print("商品描述: {}\n".format(descripList[i]))
        print("價格: \033[7;33m {} \033[0m\n".format(priceList[i]))
        print("-"*100)
        print()

輸入確定訂購的餐廳名稱: 品‧印度 Flavor of India

 ================================== 列出訂購的餐廳資訊 ==================================  

餐廳名稱:  品‧印度 Flavor of India 

類型:  店內價 咖哩 南洋料理 

評價:  4.1顆星 

運費:  29$ 運費 

超連結: https://www.foodpanda.com.tw/restaurant/y7zr/pin-yin-du-flavor-of-india

================================================== 菜單 ==================================================

商品名稱:  菠菜小煎餅 

商品描述: 含有四季豆與紅蘿蔔及菠菜製成的煎餅

價格:  NT$250.00 

----------------------------------------------------------------------------------------------------

商品名稱:  醬料蔬菜球 

商品描述: 含有花椰菜高麗菜及綜合青椒....等製成的綜合蔬菜球

價格:  NT$250.00 

----------------------------------------------------------------------------------------------------

商品名稱:  印度香脆薄餅 

商品描述: 清爽小黃瓜、番茄及洋蔥與香料搭配鹹酥脆餅

價格:  NT$100.00 

----------------------------------------------------------------------------------------------------

商品名稱:  印度黃金金三角 

商品描述: 印度香料搭配馬鈴薯與薑和香菜等製作成的經典開胃菜

價格:  NT$120.00 

-----------------------------------------------------------------

In [125]:
# 存成Excel檔
import datetime
import csv

# 現在時間格式處理
T_nowList = []
T_now = datetime.datetime.now()
T_nowFileName = T_now.strftime("%Y%m%d_%H{}%M{}%S{}").format("點","分","秒")
T_now = T_now.strftime("{}%Y{} %m{} %d{} %H:%M:%S").format("西元","年","月","日")

# 存成Excel檔
try:
    with open(StoreName+"_"+T_nowFileName+".csv", "w", newline="") as csvfile:

      # 建立 CSV 檔寫入器
        writer = csv.writer(csvfile)

      # 寫入餐廳基本資料   
        writer.writerow([T_now])
        writer.writerow(["="*50+"餐廳基本資料"+"="*50])
        writer.writerow(["餐廳名稱:",StoreName])
        writer.writerow(["類型: ",StoreCategory])
        writer.writerow(["評價: ",StoreRate])
        writer.writerow(["運費: ",StoreFee])
        writer.writerow(["超連結: ",webURL+StoreAttr])
        writer.writerow(" ")

      # 寫入一列資料
        writer.writerow(["="*50+"菜單如下"+"="*50])
        writer.writerow(['商品名稱',"","",'商品描述',"","","","","","",'價格(NT$)'])

    # 寫入另外幾列資料
        for i in range(len(nameList)):
            writer.writerow([nameList[i],"","",descripList[i],"","","","","","",priceList[i].split("$")[-1]])

    print("存檔完成!")
except:
    print("存檔過程出錯誤! 有可能是文字轉檔的問題")

存檔完成!


In [ ]:
# UBEREATS!!!!!!!!

In [126]:
# UberEats
import requests as rq
from pyquery import PyQuery as pq
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# 輸入外送地址
address = input("輸入外送地址(OO市OO區OO路OO號):")

# selenium 開啟瀏覽器
driver = webdriver.Chrome()
driver.get("https://www.ubereats.com")
# driver.maximize_window()

#找到輸入框
element = driver.find_element_by_css_selector("#location-enter-address-input")

time.sleep(0.5)

#輸入內容
element.send_keys(address)

time.sleep(1.5)

#提交表單
element.send_keys(Keys.ENTER)
# python_button = driver.find_elements_by_css_selector("#wrapper > div.ak.al.am.aj.af > div.as.ax.ay.az.ap > form > button")[0]
# python_button.click()

# time.sleep(5)

#分頁
# python_button = driver.find_element_by_xpath("//*[@id='filters-react-root']/aside/div[1]")
# python_button.click()

# element = driver.find_element_by_xpath("//*[@id='filters-react-root']/aside/div[1]/div/input")
# element.send_keys(restaurant)
# element.send_keys(Keys.ENTER)

time.sleep(5)

#抓取本頁URL並關閉瀏覽器
html = driver.find_element_by_css_selector("*").get_attribute("outerHTML")
doc = pq(html)

time.sleep(3)

driver.close()

#地址例外處理
if (doc("#wrapper > div:nth-child(2) > div > div > div").text()==""):
    print("----------------------------------------------------------------------------------------------")
    print("Oops! Something Wrong!\nTry to Key in the 'detail'(完整的) address!")

輸入外送地址(OO市OO區OO路OO號):台北市大安區辛亥路三段188號


In [127]:
# 找目前有開放訂購的餐廳
foodList = []
foodNoopenList = []
foodCategory = []
foodDict_By_Category = {}
foodDict_By_Stars = {}
foodDict_By_Fee = {}
foodDict_By_Attr = {}
foodDict_By_Time = {}
foodDict_By_NoOpen_Stars = {}

# 處理資料(EX:Name,Rate,Fee...)
for eachStore in doc("#wrapper > div:nth-child(2) > div > div > div").items():  
    # 判斷是否開放訂購外送
    if eachStore("a > div > figure > figcaption > div").text() == "":
        # {Name} = foodCategory
        foodDict = {}
        foodCategory.append(eachStore(" article.af > div > div:nth-child(2)").text())
        foodDict[eachStore("article.af > div > div:nth-child(1)").text()] = eachStore(" article.af > div > div:nth-child(2)").text()
        foodDict_By_Category.update(foodDict)
        # [Name] -> foodList
        foodList.append(eachStore("article.af > div > div:nth-child(1)").text())
        # {Name} = Rate
        foodDict = {}
        foodDict[eachStore("article.af > div > div:nth-child(1)").text()] = eachStore("article.af > div > div:nth-child(3)>div>div:nth-child(3)").text().split("(")[0]
        foodDict_By_Stars.update(foodDict)
        # {Name} = Fee
        foodDict = {}
        foodDict[eachStore("article.af > div > div:nth-child(1)").text()] = eachStore("article.af > div > div:nth-child(3)>div>div:nth-child(5)").text().split(" ")[0]
        foodDict_By_Fee.update(foodDict)
        # {Name} = attr
        foodAttr = {}
        foodAttr[eachStore("article.af > div > div:nth-child(1)").text()] = eachStore("a").attr("href")
        foodDict_By_Attr.update(foodAttr)
        # {Name} = Arrivetime
        foodTime = {}
        foodTime[eachStore("article.af > div > div:nth-child(1)").text()] = eachStore("article.af > div > div:nth-child(3)>div>div:nth-child(1)").text()
        foodDict_By_Time.update(foodTime)
        
        #處理尚未開放訂購的餐廳  {Name} = NoOpenStars
        foodDict = {}
        if eachStore("article.af > div > div:nth-child(3)>div>div:nth-child(3)").text() == "":
            name = eachStore("article.af > div > div:nth-child(1)").text()
            foodNoopenList.append(name)
        
# 由值找鍵的函式   
def get_key (dict, value):
    return [k for k, v in dict.items() if v == value]
        
# 修正TWD$放錯位置的問題
for i in range(len(foodDict_By_Stars)):
    if "TWD" in foodDict_By_Stars[foodList[i]]:
        foodDict_By_Fee[foodList[i]] =  foodDict_By_Stars[foodList[i]].split(" ")[0]
        foodDict_By_Stars[foodList[i]] = "0"

In [130]:
# 輸入想吃的種類
print("=="*20+" 以下是目前有開放訂購的餐廳 "+"=="*20)
print()

category = input("輸入想吃的食物種類(EX:美式、日式、台灣美食...): ")
print()
print("\033[1;31m =================== 目前餐廳中種類包含「"+category+"」的店家(依星星數量由多至少排序) =====================  \033[0m \n")

webURL = "https://www.ubereats.com"
results = []
Results_By_Stars ={}
category_name = []
for i in range(len(foodCategory)):
    if category in foodCategory[i]:
        # {Results_Name} = Results_rate
        category_name = get_key(foodDict_By_Category,foodCategory[i])
        if len(category_name)==1:
            Results_By_Stars[str(category_name[0])] = foodDict_By_Stars[category_name[0]]
        else:
            for j in range(len(category_name)):
                Results_By_Stars[category_name[j]] = foodDict_By_Stars[category_name[j]]
    else:
        results.append(-1)
# 例外抓取
if len(results) == len(foodCategory):
    print("可惜! 沒有包含此種類的店家!")        

# 想吃的種類中依照星星數量排序
# [Name,rate]
Results_Sorted_By_Stars = sorted(Results_By_Stars.items(),key = lambda x:x[1],reverse = True)
        
for i in range(len(Results_Sorted_By_Stars)):
    if Results_Sorted_By_Stars[i][0] in foodNoopenList:
        output = "{}. \033[7;30m {} \033[0m \033[7;37m 目前尚未開放訂購 \033[0m".format(i+1,Results_Sorted_By_Stars[i][0])
    else:
        output = "{}. \033[7;30m {} \033[0m \033[7;32m {} \033[0m  \033[7;33m {}顆星 \033[0m ".format(i+1,Results_Sorted_By_Stars[i][0],foodDict_By_Category[Results_Sorted_By_Stars[i][0]],Results_Sorted_By_Stars[i][1])
    print(output+"\n")
    print(webURL+foodDict_By_Attr[Results_Sorted_By_Stars[i][0]])
    print("-"*90)

======================================== 以下是目前有開放訂購的餐廳 ========================================

輸入想吃的食物種類(EX:美式、日式、台灣美食...): 台灣

 =================== 目前餐廳中種類包含「台灣」的店家(依星星數量由多至少排序) =====================   

1.  蛋幾ㄌㄟˇ蛋餅捲專賣店   $ • 台灣美食    4.9顆星  

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E8%9B%8B%E5%B9%BE%E3%84%8C%E3%84%9F%CB%87%E8%9B%8B%E9%A4%85%E6%8D%B2%E5%B0%88%E8%B3%A3%E5%BA%97/897C_k8xQS-oCZHdI2RNuw/
------------------------------------------------------------------------------------------
2.  師園鹽酥雞   $ • 台灣美食    4.8顆星  

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E5%B8%AB%E5%9C%92%E9%B9%BD%E9%85%A5%E9%9B%9E/7zgnlb3vRaiSRTMRNXcJ8Q/
------------------------------------------------------------------------------------------
3.  站食可以   $ • 台灣美食    4.8顆星  

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E7%AB%99%E9%A3%9F%E5%8F%AF%E4%BB%A5/bW0KckaaQKuGiwWA9aZfdw/
------------------------------------------------------------------------------------------
4.  梁鑫雞肉

In [132]:
# 輸入外送餐廳名稱
restaurant = input("輸入想外送的餐廳名稱:")
print()

# 排列最接近的Name
print("\033[1;31m =============== 目前餐廳中最接近「"+restaurant+"」的店家(依星星數量由多至少排序) ===================  \033[0m \n")

foodCloserList = []
for i in range(len(foodList)):
    if restaurant in foodList[i]:
        foodCloserList.append(foodList[i])
        
# 最接近Name的餐廳的星星數量排列
foodCloserDict_By_stars = {}
for i in range(len(foodCloserList)):
    foodCloserDict_By_stars[foodCloserList[i]] = foodDict_By_Stars[foodCloserList[i]]
foodCloserList_By_stars = sorted(foodCloserDict_By_stars.items(),key = lambda x:x[1],reverse = True)

if foodCloserList == []:
    print("可惜! "+restaurant+"目前沒有提供ubereats外送服務!")
#     print("----------------------------------------------------------------------")
#     # 找出最接近的前n個Name
#     num = input("輸入前N個最接近"+restaurant+"的結果:")
#     print()
#     for i in range(int(num)):
#         foodCloserList.append(foodList[i]) 
#         output = "{:<1d}.\t{:<20s}\t{:>10s}".format(i+1,foodCloserList[i],foodDict_By_Stars[foodList[i]])
#         print(output)

else:
    for i in range(len(foodCloserList)):
        if foodCloserList[i] in foodNoopenList:
            output = "{}. \033[7;30m {} \033[0m \033[7;37m 目前尚未開放訂購 \033[0m".format(i+1,foodCloserList_By_stars[i][0])
        else:
            output = "{}. \033[7;30m {} \033[0m \033[7;33m {}顆星 \033[0m \033[7;34m {}運費 \033[0m \033[7;36m {} \033[0m".format(i+1,foodCloserList_By_stars[i][0],foodCloserList_By_stars[i][1],foodDict_By_Fee[foodCloserList_By_stars[i][0]],foodDict_By_Time[foodCloserList_By_stars[i][0]])
        print(output)
        print()
        print(webURL+foodDict_By_Attr[foodCloserList_By_stars[i][0]])
        print("-"*90)

輸入想外送的餐廳名稱:站食可以

 =============== 目前餐廳中最接近「站食可以」的店家(依星星數量由多至少排序) ===================   

1.  站食可以   4.8顆星   40TWD運費   15 – 25 分鐘 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E7%AB%99%E9%A3%9F%E5%8F%AF%E4%BB%A5/bW0KckaaQKuGiwWA9aZfdw/
------------------------------------------------------------------------------------------


In [133]:
# 依照星星數量排列(多到少)
print("\033[1;31m ================================= 外送地址附近目前有開放訂購的餐廳(依星星數量由多至少排序) ============================== \033[0m \n")
foodDict_Sorted_By_Stars = sorted(foodDict_By_Stars.items(),key = lambda x:x[1],reverse = True)

# 輸出排序後的店家
foodDict_Sorted_Open_By_Stars = []
for i in range(len(foodDict_Sorted_By_Stars)):
    if foodDict_Sorted_By_Stars[i][0] not in foodNoopenList:
        foodDict_Sorted_Open_By_Stars.append(foodDict_Sorted_By_Stars[i])

for i in range(len(foodDict_Sorted_Open_By_Stars)):
    output = "{:<d}. \033[7;30m {} \033[0m  \033[7;33m {}顆星 \033[0m".format(i+1,foodDict_Sorted_Open_By_Stars[i][0],foodDict_Sorted_Open_By_Stars[i][1])
    print(output)
    print()
    print(webURL+foodDict_By_Attr[foodDict_Sorted_Open_By_Stars[i][0]])
    print("-"*100)
    

 ================================= 外送地址附近目前有開放訂購的餐廳(依星星數量由多至少排序) ==============================  

1.  迷客夏 延吉店    4.9顆星 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E8%BF%B7%E5%AE%A2%E5%A4%8F-%E5%BB%B6%E5%90%89%E5%BA%97/s6ipODqBQj2kdWOz5ReFSQ/
----------------------------------------------------------------------------------------------------
2.  可不可熟成紅茶 台北大安店    4.9顆星 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E5%8F%AF%E4%B8%8D%E5%8F%AF%E7%86%9F%E6%88%90%E7%B4%85%E8%8C%B6-%E5%8F%B0%E5%8C%97%E5%A4%A7%E5%AE%89%E5%BA%97/dOgUMrgcQS686sltOJsE9Q/
----------------------------------------------------------------------------------------------------
3.  蛋幾ㄌㄟˇ蛋餅捲專賣店    4.9顆星 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E8%9B%8B%E5%B9%BE%E3%84%8C%E3%84%9F%CB%87%E8%9B%8B%E9%A4%85%E6%8D%B2%E5%B0%88%E8%B3%A3%E5%BA%97/897C_k8xQS-oCZHdI2RNuw/
----------------------------------------------------------------------------------------------------
4.  可不可熟成紅茶 北醫店    4.9顆星 

h

In [134]:
# 依照運費排列(少到多)
print("\033[1;31m ================================== 外送地址附近目前有開放訂購的餐廳(依照運費由少至多排序) ================================== \033[0m \n")
foodDict_Sorted_By_Fee = sorted(foodDict_By_Fee.items(),key = lambda x:x[1],reverse = False)

# 輸出排序後的店家
foodDict_Sorted_Open_By_Fee = []
for i in range(len(foodDict_Sorted_By_Fee)):
    if foodDict_Sorted_By_Fee[i][0] not in foodNoopenList:
        foodDict_Sorted_Open_By_Fee.append(foodDict_Sorted_By_Fee[i])

for i in range(len(foodDict_Sorted_Open_By_Fee)):
    output = "{:<d}. \033[7;30m {} \033[0m   \033[7;34m {}運費 \033[0m".format(i+1,foodDict_Sorted_Open_By_Fee[i][0],foodDict_Sorted_Open_By_Fee[i][1])
    print(output)
    print()
    print(webURL+foodDict_By_Attr[foodDict_Sorted_Open_By_Fee[i][0]])
    print("-"*100)

 ================================== 外送地址附近目前有開放訂購的餐廳(依照運費由少至多排序) ==================================  

1.  麥當勞 S167台北麟光 McDonald's Lin Guang, Taipei     25TWD運費 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E9%BA%A5%E7%95%B6%E5%8B%9E-s167%E5%8F%B0%E5%8C%97%E9%BA%9F%E5%85%89-mcdonalds-lin-guang-taipei/QaNWbc88T5CGuG2k6SfUqA/
----------------------------------------------------------------------------------------------------
2.  麥當勞 S362北復興二 McDonald's Fu Sing II, Taipei     25TWD運費 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E9%BA%A5%E7%95%B6%E5%8B%9E-s362%E5%8C%97%E5%BE%A9%E8%88%88%E4%BA%8C-mcdonalds-fu-sing-ii-taipei/qc8TUvEYTVyfnRdIepM9_Q/
----------------------------------------------------------------------------------------------------
3.  老先覺麻辣窯燒鍋 台北和平三店     25TWD運費 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E8%80%81%E5%85%88%E8%A6%BA%E9%BA%BB%E8%BE%A3%E7%AA%AF%E7%87%92%E9%8D%8B-%E5%8F%B0%E5%8C%97%E5%92%8C%E5%B9%B3%E4%B8%89%E5%BA%97/hYl6q_StT3arp8K

In [135]:
# 依照到達時間排列(少到多)
print("\033[1;31m ================================== 外送地址附近目前有開放訂購的餐廳(依照到達時間由少至多排序) ================================ \033[0m \n")
foodDict_Sorted_By_Time = sorted(foodDict_By_Time.items(),key = lambda x:x[1],reverse = False)

# 輸出排序後的店家
foodDict_Sorted_Open_By_Time = []
for i in range(len(foodDict_Sorted_By_Time)):
    if foodDict_Sorted_By_Time[i][0] not in foodNoopenList:
        foodDict_Sorted_Open_By_Time.append(foodDict_Sorted_By_Time[i])
        
for i in range(len(foodDict_Sorted_Open_By_Time)):
    output = "{:<d}. \033[7;30m {} \033[0m   \033[7;36m {} \033[0m".format(i+1,foodDict_Sorted_Open_By_Time[i][0],foodDict_Sorted_Open_By_Time[i][1])
    print(output)
    print()
    print(webURL+foodDict_By_Attr[foodDict_Sorted_Open_By_Time[i][0]])
    print("-------------------------------------")

 ================================== 外送地址附近目前有開放訂購的餐廳(依照到達時間由少至多排序) ================================  

1.  可不可熟成紅茶 台北師大店     10 – 20 分鐘 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E5%8F%AF%E4%B8%8D%E5%8F%AF%E7%86%9F%E6%88%90%E7%B4%85%E8%8C%B6-%E5%8F%B0%E5%8C%97%E5%B8%AB%E5%A4%A7%E5%BA%97/UtetvqwvT9-gYPpA6ateBA/
-------------------------------------
2.  SUBWAY敦化和平店     10 – 20 分鐘 

https://www.ubereats.com/zh-TW/taipei/food-delivery/subway%E6%95%A6%E5%8C%96%E5%92%8C%E5%B9%B3%E5%BA%97/LsfhpYjzRniEYWb_o5v60g/
-------------------------------------
3.  麥當勞 S167台北麟光 McDonald's Lin Guang, Taipei     15 – 25 分鐘 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E9%BA%A5%E7%95%B6%E5%8B%9E-s167%E5%8F%B0%E5%8C%97%E9%BA%9F%E5%85%89-mcdonalds-lin-guang-taipei/QaNWbc88T5CGuG2k6SfUqA/
-------------------------------------
4.  站食可以     15 – 25 分鐘 

https://www.ubereats.com/zh-TW/taipei/food-delivery/%E7%AB%99%E9%A3%9F%E5%8F%AF%E4%BB%A5/bW0KckaaQKuGiwWA9aZfdw/
---------------------------

In [136]:
# 選定某家餐廳 
chooseRest = input("輸入確定訂購的餐廳名稱: ")
print()
if chooseRest not in foodList:
    print(" \033[1;31m OOPs! 找不到「"+chooseRest+"」，請輸入餐廳完整名稱! \033[0m")
else:
    print("\033[1;31m ================================== 列出訂購的餐廳資訊 ================================== \033[0m \n")

    # 列出該家資訊(名稱、評價、超連結、運費、菜單、價格)
    StoreName = chooseRest
    StoreRate = foodDict_By_Stars[StoreName]
    StoreFee = foodDict_By_Fee[StoreName]
    StoreTime = foodDict_By_Time[StoreName]
    StoreAttr = foodDict_By_Attr[StoreName]
    StoreCategory = foodDict_By_Category[StoreName]
    print("餐廳名稱: \033[7;30m {} \033[0m\n".format(StoreName))
    if StoreName not in foodNoopenList:
        print("類型: \033[7;32m {} \033[0m\n".format(StoreCategory))
        print("評價: \033[7;33m {}顆星 \033[0m\n".format(StoreRate))
        print("運費: \033[7;34m {} \033[0m\n".format(StoreFee))
        print("預計到達時間: \033[7;36m {} \033[0m\n".format(StoreTime))
    else:
        print("\033[7;37m 目前尚未開放訂購，可以先看菜單 \033[0m\n")
    print("超連結: {}\n".format(webURL+StoreAttr))

    # 進入該店家的網頁
    # selenium 開啟瀏覽器
    driver = webdriver.Chrome()
    driver.get(webURL+StoreAttr)
    time.sleep(1)
    #抓取本頁URL並關閉瀏覽器
    html = driver.find_element_by_css_selector("*").get_attribute("outerHTML")
    RestDoc = pq(html)
    driver.close()
#     RestResponse = rq.get(webURL+StoreAttr)
#     RestDoc = pq(RestResponse.text)

    # 爬取菜單，價格
    print("=="*25+" 菜單 "+"=="*25)
    print()

    orderList = []
    nameList = []
    descripList = []
    priceList = []
    orderDict = {}
    
# 處理資料
    for each in RestDoc("#wrapper > div:nth-child(3) > ul > li > ul > div").items():
        #處理沒有描述的商品
        if each("a > div > div > div:nth-child(3) > div").text() == "":
            #價格
            price = each("a > div > div > div:nth-child(2) > div").text()
            # 描述
            descrip = each("a > div > div > div:nth-child(1) > div").text()
        else:
            #價格
            price = each("a > div > div > div:nth-child(3) > div").text()
            # 描述
            descrip = each("a > div > div > div:nth-child(2) > div").text()
        
        # orderDict[名字] = [描述,價格]
        orderDict[each("div:nth-child(1) > h4").text()] = [descrip,price]
        orderList.append(orderDict)
        
        nameList.append(each("div:nth-child(1) > h4").text())
        descripList.append(descrip)
        priceList.append(price)
        
    if "提供" in nameList[0] :
        del nameList[0]
        del descripList[0]
        del priceList[0]

    # 輸出資料
    for i in range(len(nameList)):
        if descripList[i] == "":
            descripList[i] = "無"
        print("商品名稱: \033[7;30m {} \033[0m\n".format(nameList[i]))
        print("商品描述: {}\n".format(descripList[i]))
        print("價格: \033[7;33m {} \033[0m\n".format(priceList[i]))
        print("-"*100)
        print()

輸入確定訂購的餐廳名稱: 陸角 Omurice

 ================================== 列出訂購的餐廳資訊 ==================================  

餐廳名稱:  陸角 Omurice 

類型:  $ • 日式料理 

評價:  4.7顆星 

運費:  25TWD 

預計到達時間:  15 – 25 分鐘 

超連結: https://www.ubereats.com/zh-TW/taipei/food-delivery/%E9%99%B8%E8%A7%92-omurice/B_1G__OPQpaQcLekyLWGbg/

================================================== 菜單 ==================================================

商品名稱:  嫩煎雞腿排招牌歐姆蛋飯 Signature Omurice with Sauteed Chicken Drumstick Signature Omurice 

商品描述: 無

價格:  $210.00 

----------------------------------------------------------------------------------------------------

商品名稱:  牛五花歐姆蛋丼飯 Beef Omurice Donburi 

商品描述: 無

價格:  $150.00 

----------------------------------------------------------------------------------------------------

商品名稱:  烏梅氣泡飲 Plum Soda 

商品描述: 無

價格:  $90.00 

----------------------------------------------------------------------------------------------------

商品名稱:  冰柚氣泡飲 Grapefruit Soda 

商品描述: 無

價格:  $90.00 

---------

In [137]:
# 存成Excel檔
import datetime
import csv

# 現在時間格式處理
T_nowList = []
T_now = datetime.datetime.now()
T_nowFileName = T_now.strftime("%Y%m%d_%H{}%M{}%S{}").format("點","分","秒")
T_now = T_now.strftime("{}%Y{} %m{} %d{} %H:%M:%S").format("西元","年","月","日")

# 存成Excel檔
try:
    with open(StoreName+"_"+T_nowFileName+".csv", "w", newline="") as csvfile:

      # 建立 CSV 檔寫入器
        writer = csv.writer(csvfile)

      # 寫入餐廳基本資料   
        writer.writerow([T_now])
        writer.writerow(["="*50+"餐廳基本資料"+"="*50])
        writer.writerow(["餐廳名稱:",StoreName])
        writer.writerow(["類型: ",StoreCategory])
        writer.writerow(["評價: ",StoreRate])
        writer.writerow(["運費: ",StoreFee])
        writer.writerow(["超連結: ",webURL+StoreAttr])
        writer.writerow(" ")

      # 寫入一列資料
        writer.writerow(["="*50+"菜單如下"+"="*50])
        writer.writerow(['商品名稱',"","","",'商品描述',"","","","","","",'價格'])

    # 寫入另外幾列資料
        for i in range(len(nameList)):
            writer.writerow([nameList[i],"","","",descripList[i],"","","","","","",priceList[i]])

    print("存檔完成!")
except:
    print("存檔過程出錯誤! 有可能是文字轉檔的問題")

存檔完成!
